# Pruebas

Buenas!

Como verás, armé un repo nuevo pero esta vez manejé mejor los archivos y quedó más prolijo (tuve problemas con el otro repo y me cansé de pelear con eso).

Considerando que había errores en la forma en la que estaba obteniendo los vectores de las emociones, decidí hacer algunas pruebas en un notebook, que es un poco más dinámico. Como es solo para hacer pruebas, estoy usando solo regresión lineal, podria intentar lo mismo con PCA, pero primero me gustaría saber si tiene sentido lógico lo que estoy haciendo.

Por favor, me gustaría que le pegues una leída al código, para ver si te parece que lo que estoy haciendo tiene sentido. Sentite libre de hacer cualquier modificación. Esto debería andar bien en otras compus, pero si no anda avisame y te paso acceso a mi usuario.

## Definiciones

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression

def load_latent_vector(name):
    """Carga el vector latente desde un archivo .npz basado en el nombre de la imagen."""
    path = f'./images/results_BU_3DFE/{name}/projected_w.npz'
    if not os.path.exists(path):
        raise FileNotFoundError(f"El archivo {path} no existe.")
    data = np.load(path)
    return data['w']

def normalize_vectors(vectors):
    """Normaliza una lista de vectores usando norma L2."""
    original_shape = vectors.shape  # Guardamos la forma original
    vectors = vectors.reshape(vectors.shape[0], -1)  # Aplanamos a (N, D)
    vectors = normalize(vectors, axis=1)  # Normalizamos en la dimensión correcta
    return vectors.reshape(original_shape)  # Restauramos la forma original

def method_1_average_then_regression(df):
    """Método 1: Promedia los vectores por emoción y aplica regresión."""
    results = {}
    emotions = ['DI', 'HA', 'SU', 'AN', 'SA', 'FE']
    
    for emotion in emotions:
        subset = df[df['exp'] == emotion]
        vectors = np.array([load_latent_vector(name) for name in subset['name']])
        vectors = normalize_vectors(vectors)
        avg_vector = np.mean(vectors, axis=0)
        
        X = np.arange(len(vectors)).reshape(-1, 1)
        y = vectors.reshape(vectors.shape[0], -1)  # Aplanamos los vectores antes de la regresión

        model = LinearRegression().fit(X, y)
        results[emotion] = model.coef_.reshape(1, 18, 512)  # Restauramos la forma original
    
    return results

def method_2_regression_by_emotion_and_level(df):
    """Método 2: Aplica regresión a cada emoción y nivel de intensidad."""
    results = {}
    emotions = ['DI', 'HA', 'SU', 'AN', 'SA', 'FE']
    
    for emotion in emotions:
        results[emotion] = {}  # Ahora almacenamos los resultados por nivel
        
        for level in sorted(df['exp_level'].unique()):
            subset = df[(df['exp'] == emotion) & (df['exp_level'] == level)]
            if subset.empty:
                continue
            vectors = np.array([load_latent_vector(name) for name in subset['name']])
            vectors = normalize_vectors(vectors)
            
            X = np.arange(len(vectors)).reshape(-1, 1)
            y = vectors.reshape(vectors.shape[0], -1)  # Aplanamos los vectores antes de la regresión
            
            model = LinearRegression().fit(X, y)
            results[emotion][level] = model.coef_.reshape(1, 18, 512)  # Guardamos cada nivel separadamente
    
    return results

def method_3_regression_with_level_variable(df):
    """Método 3: Incluye el nivel como variable numérica en la regresión."""
    results = {}
    emotions = ['DI', 'HA', 'SU', 'AN', 'SA', 'FE']
    
    for emotion in emotions:
        subset = df[df['exp'] == emotion]
        vectors = np.array([load_latent_vector(name) for name in subset['name']])
        vectors = normalize_vectors(vectors)
        levels = subset['exp_level'].values.reshape(-1, 1)

        y = vectors.reshape(vectors.shape[0], -1)
        
        model = LinearRegression().fit(levels, y)
        results[emotion] = model.coef_.reshape(1, 18, 512)
    
    return results

def save_results_as_csv(results, filename):
    flat_results = {}
    
    for key, value in results.items():
        flat_results[key] = value.flatten()  # Aplanar la matriz
    
    df = pd.DataFrame.from_dict(flat_results, orient='index')
    df.to_csv(filename)

    save_results_as_csv(method_1_results, 'method_1_results.csv')


def save_results_as_npz(results, filename):
    """Guarda los resultados en un archivo NPZ."""
    np.savez(filename, **results)


## Cargar los datos

In [2]:
# Cargar datos y ejecutar procesos
df = pd.read_csv('./dataframes/processed_dataframe_combined_fallback.csv')

# Cuando procesé las imágenes por primera vez, para algunos ids no pude procesar
# todas las emociones, así que elimino esos ids para que no causen problemas
df['idUnique'] = df['id'].astype(str) + df['gender']
ids_malos = ["39M", "17M", "22M", "14M", "2F"] 
df = df[~df['idUnique'].isin(ids_malos)]

## Ejecutar los 3 métodos y guardar como npz

In [3]:
method_1_results = method_1_average_then_regression(df)
method_2_results = method_2_regression_by_emotion_and_level(df)
method_3_results = method_3_regression_with_level_variable(df)

print("Imprimo shape de uno de los vectores para controlar formato:")
print("Resultados método 1, 'DI': " + str(method_1_results['DI'].shape))
print("Resultados método 2, 'DI': " + str(method_2_results['DI'][1].shape))
print("Resultados método 3, 'DI': " + str(method_3_results['DI'].shape))

# Guardar resultados
save_results_as_npz(method_1_results, 'method_1_results.npz')
save_results_as_npz(method_2_results, 'method_2_results.npz')
save_results_as_npz(method_3_results, 'method_3_results.npz')

Imprimo shape de uno de los vectores para controlar formato:
Resultados método 1, 'DI': (1, 18, 512)
Resultados método 2, 'DI': (1, 18, 512)
Resultados método 3, 'DI': (1, 18, 512)


## Printeo la forma de los diccionarios para saber con qué estoy trabajando

In [4]:
def print_shape(diccionario, indent=0):
    for clave, valor in diccionario.items():
        # Indentación para la estructura
        print("  " * indent + f"Clave: {clave}", end=' ')
        
        if isinstance(valor, dict):
            # Si el valor es otro diccionario, lo recorremos recursivamente
            print("; Valor: Diccionario con las siguientes claves: ")
            print_shape(valor, indent + 1)
        elif isinstance(valor, np.ndarray):
            # Si el valor es un array de NumPy, mostramos su forma
            print(f"; Valor: Array de la forma {valor.shape}")
        elif isinstance(valor, list):
            # Si el valor es una lista, mostramos su longitud
            print(f"(Lista) - Shape: {len(valor)}")
        else:
            # Si no es ni diccionario, ni array ni lista, solo mostramos el tipo
            print(f"({type(valor).__name__})")

print("Forma del diccionario correspondiente al método 1: ")
print_shape(method_1_results)
print("Fin del diccionario correspondiente al método 1.\n")
print("Forma del diccionario correspondiente al método 2: ")
print_shape(method_2_results)
print("Fin del diccionario correspondiente al método 2.\n")
print("Forma del diccionario correspondiente al método 3: ")
print_shape(method_3_results)
print("Fin del diccionario correspondiente al método 3.\n")

Forma del diccionario correspondiente al método 1: 
Clave: DI ; Valor: Array de la forma (1, 18, 512)
Clave: HA ; Valor: Array de la forma (1, 18, 512)
Clave: SU ; Valor: Array de la forma (1, 18, 512)
Clave: AN ; Valor: Array de la forma (1, 18, 512)
Clave: SA ; Valor: Array de la forma (1, 18, 512)
Clave: FE ; Valor: Array de la forma (1, 18, 512)
Fin del diccionario correspondiente al método 1.

Forma del diccionario correspondiente al método 2: 
Clave: DI ; Valor: Diccionario con las siguientes claves: 
  Clave: 1 ; Valor: Array de la forma (1, 18, 512)
  Clave: 2 ; Valor: Array de la forma (1, 18, 512)
  Clave: 3 ; Valor: Array de la forma (1, 18, 512)
  Clave: 4 ; Valor: Array de la forma (1, 18, 512)
Clave: HA ; Valor: Diccionario con las siguientes claves: 
  Clave: 1 ; Valor: Array de la forma (1, 18, 512)
  Clave: 2 ; Valor: Array de la forma (1, 18, 512)
  Clave: 3 ; Valor: Array de la forma (1, 18, 512)
  Clave: 4 ; Valor: Array de la forma (1, 18, 512)
Clave: SU ; Valor: D

# Intentando analisis de datos

## 1. Similitud coseno

In [5]:
from scipy.spatial.distance import cosine
import numpy as np

# Función para calcular la similitud coseno entre dos vectores
def cosine_similarity(v1, v2):
    return 1 - cosine(v1.flatten(), v2.flatten())

In [6]:
# Función para comparar método 1 vs método 3
def compare_results(results1, results3):
    similarities = {}
    for emotion in results1.keys():
        similarities[emotion] = cosine_similarity(results1[emotion], results3[emotion])
    return similarities

In [7]:
print(compare_results(method_1_results, method_3_results))

{'DI': 0.11721087677114694, 'HA': 0.17827751891230292, 'SU': -0.013326488144749016, 'AN': -0.12284548334796241, 'SA': 0.16511259676313994, 'FE': 0.2575398268295501}


### Problema: Me da mal la similitud coseno, cómo sé cuál método es correcto/mejor?
Generar imágenes con estos vectores no es una opción porque no anda el docker.